In [2]:
!pip install pulp

In [3]:
from pulp import * 
import pandas as pd
df = pd.read_csv("NutritionalFacts_Fruit_Vegetables_Seafood.csv", encoding='latin-1')
df.head()

,Food and Serving,Calories,CaloriesÊfrom Fat,Total Fat,Total Fat.1,Sodium,Sodium.1,Potassium,Potassium.1,Total Carbo-hydrate,...,Protein,Vitamin A,Vitamin C,Calcium,ÊÊIronÊÊ,Saturated Fat,Saturated Fat.1,Chole-sterol,Chole-sterol.1,Food Type
0,NaN,NaN,NaN,(g),(%DV),(g),(%DV),(g),(%DV),(g),...,(g),(%DV),(%DV),(%DV),(%DV),(%DV),(mg)Ê,(%DV),(mg)Ê,NaN
1,"Asparagus, 5 spears (93 g/3.3 oz)",20.0,0.0,0,0,0,0,230,7,4,...,2,10,15,2,2,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Êounce w..."
2,"Bell Pepper, 1 medium (148 g/5.3 oz)",25.0,0.0,0,0,40,2,220,6,6,...,1,4,190,2,4,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Êounce w..."
3,"Broccoli, 1 medium stalk (148 g/5.3 oz)",45.0,0.0,0.5,1,80,3,460,13,8,...,4,6,220,6,6,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Êounce w..."
4,"Carrot, 1 carrot, 7"" long,Ê1 1/4"" diameter (78...",30.0,0.0,0,0,60,3,250,7,7,...,1,110,10,2,2,NaN,NaN,NaN,NaN,"Vegetables, Serving Size (gram weight/Êounce w..."


In [4]:
dfsmall=df.filter(items=['Food and Serving','Sodium.1', 'Potassium.1', 'Calcium'], axis=1).filter(items=range(1,9), axis=0)
dfsmall.columns=('Food','Sodium','Potassium','Calcium')
dfsmall

,Food,Sodium,Potassium,Calcium
1,"Asparagus, 5 spears (93 g/3.3 oz)",0,7,2
2,"Bell Pepper, 1 medium (148 g/5.3 oz)",2,6,2
3,"Broccoli, 1 medium stalk (148 g/5.3 oz)",3,13,6
4,"Carrot, 1 carrot, 7"" long,Ê1 1/4"" diameter (78...",3,7,2
5,"Cauliflower, 1/6 medium head (99 g/3.5 oz)",1,8,2
6,"Celery, 2 medium stalks (110 g/3.9 oz)",5,7,4
7,"Cucumber, 1/3 medium (99 g/3.5 oz)",0,4,2
8,"Green (Snap) Beans, 3/4 cup cut (83 g/3.0 oz)",0,6,4


In [5]:
short_names = dfsmall['Food'].tolist()
short_names=[elem[:5] for elem in short_names]
short_names




['Aspar', 'Bell ', 'Brocc', 'Carro', 'Cauli', 'Celer', 'Cucum', 'Green']

In [6]:
for i in range(8): 
    dfsmall.at[i+1,'Food'] = short_names[i]

In [7]:
dfsmall

,Food,Sodium,Potassium,Calcium
1,Aspar,0,7,2
2,Bell,2,6,2
3,Brocc,3,13,6
4,Carro,3,7,2
5,Cauli,1,8,2
6,Celer,5,7,4
7,Cucum,0,4,2
8,Green,0,6,4


In [8]:
prob = LpProblem(name="fruits", sense=LpMinimize)
#items=(name for name in dfsmall['Food'])
foods = pulp.LpVariable.dicts('food',(name for name in dfsmall['Food']),
                                   lowBound=0, upBound=5,
                                   cat='Continuous')


In [9]:
foods

{'Aspar': food_Aspar,
 'Bell ': food_Bell_,
 'Brocc': food_Brocc,
 'Carro': food_Carro,
 'Cauli': food_Cauli,
 'Celer': food_Celer,
 'Cucum': food_Cucum,
 'Green': food_Green}

In [14]:
dfsmall.loc[1,'Food']

'Aspar'

In [10]:
#add objective
prob += lpSum([foods[dfsmall.loc[i,'Food']]*int(dfsmall.loc[i,'Sodium']) for i in dfsmall.index])

#add contraints
prob += lpSum([foods[dfsmall.loc[i,'Food']]*int(dfsmall.loc[i,'Potassium']) for i in dfsmall.index]) >=70
prob += lpSum([foods[dfsmall.loc[i,'Food']]*int(dfsmall.loc[i,'Calcium']) for i in dfsmall.index]) >=55

prob

fruits:
MINIMIZE
2*food_Bell_ + 3*food_Brocc + 3*food_Carro + 1*food_Cauli + 5*food_Celer + 0
SUBJECT TO
_C1: 7 food_Aspar + 6 food_Bell_ + 13 food_Brocc + 7 food_Carro + 8 food_Cauli
 + 7 food_Celer + 4 food_Cucum + 6 food_Green >= 70

_C2: 2 food_Aspar + 2 food_Bell_ + 6 food_Brocc + 2 food_Carro + 2 food_Cauli
 + 4 food_Celer + 2 food_Cucum + 4 food_Green >= 55

VARIABLES
food_Aspar <= 5 Continuous
food_Bell_ <= 5 Continuous
food_Brocc <= 5 Continuous
food_Carro <= 5 Continuous
food_Cauli <= 5 Continuous
food_Celer <= 5 Continuous
food_Cucum <= 5 Continuous
food_Green <= 5 Continuous

In [11]:
prob.solve()
LpStatus[prob.status]

'Optimal'

In [12]:
for i in foods:
    print('Amount of ', foods[i], ': ', foods[i].varValue)

Amount of  food_Aspar :  5.0
Amount of  food_Bell_ :  0.0
Amount of  food_Brocc :  2.5
Amount of  food_Carro :  0.0
Amount of  food_Cauli :  0.0
Amount of  food_Celer :  0.0
Amount of  food_Cucum :  5.0
Amount of  food_Green :  5.0


In [13]:
value(prob.objective)

7.5